This notebook is the process by which all of the database are created to drive the "discover" portion of the dashboard.

There are generic names for the things being discovered:

  - Stations with Wind Stress
  - Stations with Net Surface Heat Flux
  - Stations with Bulk Turbulent Heat Flux
  - Stations with Precipitation
  - Stations with Evaporation minus Precipitation

But within each data set the variable which are related to those topics have different names. For example Bulk Turbulent Heat Flux is related to QLAT and QSEN in some data sets and QL and QS in others. The counts are separated into database tables according to the variables that make up the "discovery" question. So for bulk Turbulent heat transfer there is a QLAT_QSEN table and a QL_QS table. The table contains the ERDDAP data set ID that contains the data.

In [6]:
import pandas as pd
import json
import constants
from sdig.erddap.info import Info
import numpy as np
from functools import reduce
import urllib

In [9]:
with open("flux_discovery.json") as discovery_stream:
    discovery_json = json.load(discovery_stream)

In [37]:
discovery = discovery_json['discovery']
for q in discovery:
    searches = discovery[q]['search']
    for search in searches:
        short_names = search['short_names']
        table = '_'.join(short_names)
        d0 = None
        for source in search['datasets']:
            did =  source[source.rindex('/') + 1:]
            site_url = source + '.csv?site_code&distinct()'
            site_df = pd.read_csv(site_url, skiprows=[1])
            to_get = ','.join(short_names)
            for site in list(site_df['site_code']):
                con = urllib.parse.quote(f'&site_code="{site}"&orderByCount(\"site_code,wmo_platform_code,time/1month\")')
                count_url = f'{source}.csv?{to_get},time,wmo_platform_code,site_code{con}'
                df = pd.read_csv(count_url, skiprows=[1])
                df['did'] = did
                if d0 is None:
                    d0 = df
                else:
                    d0 = pd.concat([d0, df])
        d0.to_csv(f'counts/nobs_{table}.csv', index=False)
        with constants.postgres_engine.connect() as conn:
            d0.to_sql(f'nobs_{table}', index=False, con=conn, if_exists='replace')    


In [6]:
platform_file = 'oceansites_flux_list.json'
platform_json = None
if platform_file is not None:
    with open(platform_file) as platform_stream:
        platform_json = json.load(platform_stream)

In [7]:
variables_by_did = {}
locations_by_did = {}
units_by_did = {}
metadata_by_did = {}
loc_df = None
plats = platform_json['config']['datasets']
for dataset in plats:
    url = dataset['url']
    locations_url = dataset['locations']
    did = url[url.rindex('/') + 1:]
    dataset['id'] = did
    info = Info(url)
    title = info.get_title()
    dataset['title'] = title
    start_date, end_date, start_date_seconds, end_date_seconds = info.get_times()
    dataset['start_date'] = start_date
    dataset['end_date'] = end_date
    dataset['start_date_seconds'] = start_date_seconds
    dataset['end_date_seconds'] = end_date_seconds
    variables_list, long_names, units, standard_names, d_types = info.get_variables()
    units_by_did[did] = units
    variables_by_did[did] = variables_list
    metadata_by_did[did] = dataset
    mdf = pd.read_csv(locations_url, skiprows=[1],
                      dtype={'wmo_platform_code': str, 'site_code': str, 'latitude': np.float64, 'longitude': np.float64})
    if mdf.shape[0] > 1 and mdf.site_code.nunique() <= 1:
        adf = mdf.mean(axis=0, numeric_only=True)
        print(adf)
        adf['site_code'] = mdf['site_code'].iloc[0]
        adf['wmo_platform_code'] = mdf['wmo_platform_code'].iloc[0]
        mdf = pd.DataFrame(columns=['latitude', 'longitude', 'site_code', 'wmo_platform_code'], index=[0], )
        mdf['latitude'] = adf.loc['latitude']
        mdf['longitude'] = adf.loc['longitude']
        mdf['site_code'] = adf.loc['site_code']
    if loc_df is None:
        loc_df = mdf
    else:
        loc_df = pd.concat([loc_df, mdf])
    

latitude     14.777487
longitude   -50.920884
dtype: float64


In [16]:
loc_df = loc_df.drop_duplicates()
with constants.postgres_engine.connect() as conn:
    loc_df.to_sql('locations', index=False, con=conn, if_exists='replace')
loc_df

,site_code,wmo_platform_code,latitude,longitude
0,0n0e,13010,0.000000,0.000000
1,0n10w,15002,0.000000,-10.000000
2,0n110w,32323,0.000000,-110.000000
3,0n125w,51011,0.000000,-125.000000
4,0n137e,52081,0.000000,137.000000
...,...,...,...,...
1,Stratus,38400.0,-20.000000,-85.000000
2,WHOTS,51400.0,22.700000,-158.000000
0,32n145e,28401,32.000000,145.000000
0,50n145w,48400,50.000000,-145.000000


In [17]:
ddf = pd.DataFrame.from_dict(metadata_by_did, orient='index')
with constants.postgres_engine.connect() as conn:
    ddf.to_sql('metadata', index=False, con=conn, if_exists='replace')
ddf

,url,locations,id,title,start_date,end_date,start_date_seconds,end_date_seconds
tao_flux_clim_lwr_absolute,https://data.pmel.noaa.gov/pmel/erddap/tableda...,https://data.pmel.noaa.gov/pmel/erddap/tableda...,tao_flux_clim_lwr_absolute,TAO OceanSITES flux data Absolute Wind Speed a...,1997-06-19,2021-09-28,8.667180e+08,1.632852e+09
tao_flux_clim_lwr_relative,https://data.pmel.noaa.gov/pmel/erddap/tableda...,https://data.pmel.noaa.gov/pmel/erddap/tableda...,tao_flux_clim_lwr_relative,TAO OceanSITES flux data Relative Wind Speed a...,1998-03-09,2021-09-28,8.894178e+08,1.632852e+09
tao_flux_absolute,https://data.pmel.noaa.gov/pmel/erddap/tableda...,https://data.pmel.noaa.gov/pmel/erddap/tableda...,tao_flux_absolute,TAO OceanSITES flux data with LWR and Absolute...,2000-04-22,2021-09-07,9.563976e+08,1.631038e+09
tao_flux_relative,https://data.pmel.noaa.gov/pmel/erddap/tableda...,https://data.pmel.noaa.gov/pmel/erddap/tableda...,tao_flux_relative,TAO OceanSITES flux data with LWR and Relative...,2000-04-26,2021-09-07,9.567324e+08,1.631038e+09
WHOI_flux,https://data.pmel.noaa.gov/pmel/erddap/tableda...,https://data.pmel.noaa.gov/pmel/erddap/tableda...,WHOI_flux,Surface fluxes calculated with Coare 3.0 algor...,2000-10-08,2018-09-25,9.709686e+08,1.537889e+09
ocs_keo_flux,https://data.pmel.noaa.gov/pmel/erddap/tableda...,https://data.pmel.noaa.gov/pmel/erddap/tableda...,ocs_keo_flux,OceanSITES Keo Flux hourly data,2004-06-16,2020-05-19,1.087416e+09,1.589857e+09
ocs_papa_flux,https://data.pmel.noaa.gov/pmel/erddap/tableda...,https://data.pmel.noaa.gov/pmel/erddap/tableda...,ocs_papa_flux,OceanSITES Papa Flux hourly data,2007-06-08,2022-02-24,1.181279e+09,1.645722e+09
Stratus_met,https://data.pmel.noaa.gov/pmel/erddap/tableda...,https://data.pmel.noaa.gov/pmel/erddap/tableda...,Stratus_met,OceanSITES Surface meteorology from the Stratu...,2000-10-08,2018-04-08,9.709686e+08,1.523191e+09
WHOTS_met,https://data.pmel.noaa.gov/pmel/erddap/tableda...,https://data.pmel.noaa.gov/pmel/erddap/tableda...,WHOTS_met,"OceanSITES Surface meteorology from the WHOI, ...",2004-08-13,2018-09-25,1.092371e+09,1.537889e+09
NTAS_met,https://data.pmel.noaa.gov/pmel/erddap/tableda...,https://data.pmel.noaa.gov/pmel/erddap/tableda...,NTAS_met,OceanSITES Surface meteorology from the Northw...,2001-03-31,2018-06-12,9.859986e+08,1.528803e+09


In [18]:
udf = pd.DataFrame.from_dict(units_by_did, orient='index')
udf = udf.reset_index().rename(columns={"index":"did"})
with constants.postgres_engine.connect() as conn:
    udf.to_sql('units', index=False, if_exists='replace', con=conn)
udf

,did,time,latitude,longitude,HEIGHT,HEIGHTZS,QLAT,QSEN,QRAIN,SWNET,...,ATMS,LW,SW,UWND,VWND,TEMP,PSAL,CNDC,UCUR,VCUR
0,tao_flux_clim_lwr_absolute,seconds since 1970-01-01T00:00:00Z,degrees_north,degrees_east,meters,meters,W m-2,W m-2,W m-2,W m-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tao_flux_clim_lwr_relative,seconds since 1970-01-01T00:00:00Z,degrees_north,degrees_east,meters,meters,W m-2,W m-2,W m-2,W m-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tao_flux_absolute,seconds since 1970-01-01T00:00:00Z,degrees_north,degrees_east,meters,meters,W m-2,W m-2,W m-2,W m-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tao_flux_relative,seconds since 1970-01-01T00:00:00Z,degrees_north,degrees_east,meters,meters,W m-2,W m-2,W m-2,W m-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WHOI_flux,seconds since 1970-01-01T00:00:00Z,degrees_north,degrees_east,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ocs_keo_flux,seconds since 1970-01-01T00:00:00Z,degrees_north,degrees_east,meters,NaN,W m-2,W m-2,W m-2,W m-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ocs_papa_flux,seconds since 1970-01-01T00:00:00Z,degrees_north,degrees_east,meters,NaN,W m-2,W m-2,W m-2,W m-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Stratus_met,seconds since 1970-01-01T00:00:00Z,degrees_north,degrees_east,NaN,NaN,NaN,NaN,NaN,NaN,...,millibars,W m-2,W m-2,meters/second,meters/second,degree_C,.001,S m-1,S m-1,S m-1
8,WHOTS_met,seconds since 1970-01-01T00:00:00Z,degrees_north,degrees_east,NaN,NaN,NaN,NaN,NaN,NaN,...,millibars,W m-2,W m-2,meters/second,meters/second,degree_C,.001,S m-1,S m-1,S m-1
9,NTAS_met,seconds since 1970-01-01T00:00:00Z,degrees_north,degrees_east,NaN,NaN,NaN,NaN,NaN,NaN,...,millibars,W m-2,W m-2,meters/second,meters/second,degree_C,.001,S m-1,S m-1,S m-1


In [10]:
discovery = discovery_json
discovery_df = None
query='site_code&distinct()'
discovery_expanded = {}
for discovery_id in discovery['discovery']:
    discovery_expanded[discovery_id] = {}
    question = discovery['discovery'][discovery_id]
    for collection in question['search']:
        short_string = ','.join(collection['short_names'])
        for url in collection['datasets']:
            did = url[url.rfind("/")+1:]
            r_url = url+'.csv?'+query
            df = pd.read_csv(r_url, skiprows=[1])
            df['question_id'] = discovery_id
            df['question_title'] = question['question']
            df['short_string'] = short_string
            df['did'] = did
            if discovery_df is None:
                discovery_df = df
            else:
                discovery_df = pd.concat([discovery_df, df])
with constants.postgres_engine.connect() as conn:
    discovery_df.to_sql('discovery', index=False, if_exists='replace', con=conn)
discovery_df

,site_code,question_id,question_title,short_string,did
0,0n10w,wind_stress,Stations with Wind Stress,"TAUX,TAUY",tao_flux_clim_lwr_relative
1,0n110w,wind_stress,Stations with Wind Stress,"TAUX,TAUY",tao_flux_clim_lwr_relative
2,0n125w,wind_stress,Stations with Wind Stress,"TAUX,TAUY",tao_flux_clim_lwr_relative
3,0n137e,wind_stress,Stations with Wind Stress,"TAUX,TAUY",tao_flux_clim_lwr_relative
4,0n140w,wind_stress,Stations with Wind Stress,"TAUX,TAUY",tao_flux_clim_lwr_relative
...,...,...,...,...,...
93,8s80.5e,evap_minus_precip,Stations with Evaporation minus Precipitation,"RAIN,EVAP",tao_flux_clim_lwr_absolute
94,8s95e,evap_minus_precip,Stations with Evaporation minus Precipitation,"RAIN,EVAP",tao_flux_clim_lwr_absolute
95,8s95w,evap_minus_precip,Stations with Evaporation minus Precipitation,"RAIN,EVAP",tao_flux_clim_lwr_absolute
0,32n145e,evap_minus_precip,Stations with Evaporation minus Precipitation,"RAIN,EVAP",ocs_keo_flux


In [39]:
unroll = []
for did in variables_by_did:
    for short_name in variables_by_did[did]:
        unroll.append({'did': did, 'short_name': short_name})
vdf = pd.DataFrame(unroll)
with constants.postgres_engine.connect() as conn:
    vdf.to_sql('variables', index=False, if_exists='replace', con=conn)
vdf

NameError: name 'variables_by_did' is not defined